In [ ]:
import os
import requests
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoImageProcessor, AutoModelForImageClassification, Trainer, TrainingArguments
)
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score
from io import BytesIO
from torchvision import transforms

os.environ["WANDB_DISABLED"] = "true"

# Experimentation with different models and configurations
models_to_try = [
    "microsoft/swin-tiny-patch4-window7-224",
    "facebook/dino-vitb8",
    "Falah/Alzheimer_classification_model",
    "google/vit-base-patch16-224",
]

# Load datasets
urltest = "https://huggingface.co/datasets/Falah/Alzheimer_MRI/resolve/main/data/test-00000-of-00001-44110b9df98c5585.parquet"
urltrain = "https://huggingface.co/datasets/Falah/Alzheimer_MRI/resolve/main/data/train-00000-of-00001-c08a401c53fe5312.parquet"
output_file_test = "test_data.parquet"
output_file_train = "train_data.parquet"

def download_file(url, output_file):
    if not os.path.exists(output_file):
        print(f"Downloading {output_file}...")
        response = requests.get(url)
        with open(output_file, 'wb') as f:
            f.write(response.content)
        print("Download complete")
    else:
        print(f"{output_file} already exists.")

download_file(urltrain, output_file_train)
download_file(urltest, output_file_test)

data_train = pd.read_parquet(output_file_train)
data_test = pd.read_parquet(output_file_test)

class AlzheimerDataset(Dataset):
    def __init__(self, dataframe, processor):
        self.data = dataframe
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_data = self.data.iloc[idx]['image']['bytes']
        label = self.data.iloc[idx]['label']
        image = Image.open(BytesIO(image_data)).convert("RGB")
        encoding = self.processor(images=image, return_tensors="pt")
        return {
            "pixel_values": encoding["pixel_values"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

seed = 42
np.random.seed(seed)

for model_name in models_to_try:
    print(f"\nExperimenting with model: {model_name}")

    processor = AutoImageProcessor.from_pretrained(model_name)

    train_dataset = AlzheimerDataset(data_train, processor)
    train_dataset_subsample_indices = np.random.choice(len(train_dataset), size=200, replace=False)
    train_dataset_subsample = torch.utils.data.Subset(train_dataset, train_dataset_subsample_indices)

    test_dataset = AlzheimerDataset(data_test, processor)
    test_dataset_subsample_indices = np.random.choice(len(test_dataset), size=50, replace=False)
    test_dataset_subsample = torch.utils.data.Subset(test_dataset, test_dataset_subsample_indices)

    model = AutoModelForImageClassification.from_pretrained(model_name, num_labels=4, ignore_mismatched_sizes=True)

    for param in model.base_model.parameters():
        param.requires_grad = False

    training_args = TrainingArguments(
        output_dir=f"./results_{model_name}",
        evaluation_strategy="no",
        save_strategy="no",
        learning_rate=5e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        weight_decay=0.01,
        logging_dir=f"./logs_{model_name}",
        logging_steps=20,
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        fp16=True,
        gradient_accumulation_steps=2,
    )

    def compute_metrics(pred):
        labels = pred.label_ids
        preds = np.argmax(pred.predictions, axis=1)
        acc = accuracy_score(labels, preds)
        f1 = f1_score(labels, preds, average="weighted")
        return {"accuracy": acc, "f1": f1}

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset_subsample,
        eval_dataset=test_dataset_subsample,
        tokenizer=processor,
        compute_metrics=compute_metrics
    )

    trainer.train()

    metrics = trainer.evaluate(test_dataset_subsample)
    print(f"Test Metrics for {model_name}:", metrics)


train_data.parquet already exists.
test_data.parquet already exists.

Experimenting with model: microsoft/swin-tiny-patch4-window7-224


config.json:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/113M [00:00<?, ?B/s]

Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-tiny-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-2-955beb6da603>:

Step,Training Loss
20,2.855000


...

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-2-955beb6da603>:117: FutureW

Step,Training Loss
20,2.568100


Test Metrics for google/vit-base-patch16-224: {'eval_loss': 1.1340017318725586, 'eval_accuracy': 0.6, 'eval_f1': 0.45, 'eval_runtime': 33.5279, 'eval_samples_per_second': 1.491, 'eval_steps_per_second': 0.388, 'epoch': 1.0}


In [ ]:
import os
import requests
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoImageProcessor, AutoModelForImageClassification, Trainer, TrainingArguments
)
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score
from io import BytesIO
from torchvision import transforms

os.environ["WANDB_DISABLED"] = "true"

models_to_try = [
    "Falah/Alzheimer_classification_model"
]

urltest = "https://huggingface.co/datasets/Falah/Alzheimer_MRI/resolve/main/data/test-00000-of-00001-44110b9df98c5585.parquet"
urltrain = "https://huggingface.co/datasets/Falah/Alzheimer_MRI/resolve/main/data/train-00000-of-00001-c08a401c53fe5312.parquet"
output_file_test = "test_data.parquet"
output_file_train = "train_data.parquet"

def download_file(url, output_file):
    if not os.path.exists(output_file):
        print(f"Downloading {output_file}...")
        response = requests.get(url)
        with open(output_file, 'wb') as f:
            f.write(response.content)
        print("Download complete")
    else:
        print(f"{output_file} already exists.")

download_file(urltrain, output_file_train)
download_file(urltest, output_file_test)

data_train = pd.read_parquet(output_file_train)
data_test = pd.read_parquet(output_file_test)

class AlzheimerDataset(Dataset):
    def __init__(self, dataframe, processor):
        self.data = dataframe
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_data = self.data.iloc[idx]['image']['bytes']
        label = self.data.iloc[idx]['label']
        image = Image.open(BytesIO(image_data)).convert("RGB")
        encoding = self.processor(images=image, return_tensors="pt")
        return {
            "pixel_values": encoding["pixel_values"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

def subset_data(dataframe, size):
    return dataframe.sample(n=min(size, len(dataframe)))

def train_and_evaluate_quick(model_name, learning_rate):
    print(f"\nQuick Training {model_name} with lr={learning_rate}")

    processor = AutoImageProcessor.from_pretrained(model_name)

    train_subset = subset_data(data_train, 250)
    test_subset = subset_data(data_test, 100)

    train_dataset = AlzheimerDataset(train_subset, processor)
    test_dataset = AlzheimerDataset(test_subset, processor)

    model = AutoModelForImageClassification.from_pretrained(model_name, num_labels=4, ignore_mismatched_sizes=True)

    for param in model.base_model.parameters():
        param.requires_grad = False

    training_args = TrainingArguments(
        output_dir=f"./results_quick_{model_name.replace('/', '_')}",
        evaluation_strategy="epoch",
        save_strategy="no",
        learning_rate=learning_rate,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        weight_decay=0.01,
        logging_dir=f"./logs_quick_{model_name.replace('/', '_')}",
        logging_steps=100,
        fp16=True,
    )

    def compute_metrics(pred):
        labels = pred.label_ids
        preds = np.argmax(pred.predictions, axis=1)
        acc = accuracy_score(labels, preds)
        f1 = f1_score(labels, preds, average="weighted")
        return {"accuracy": acc, "f1": f1}

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=processor,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    metrics = trainer.evaluate(test_dataset)
    print(f"Quick Metrics for {model_name} with lr={learning_rate}: {metrics}")
    return metrics

random_hyperparameter = [
    {"learning_rate": 1e-4},
    {"learning_rate": 5e-5},
    {"learning_rate": 1e-5},
]

quick_results = []

for model_name in models_to_try:
    for combo in random_hyperparameter:
        metrics = train_and_evaluate_quick(
            model_name,
            combo["learning_rate"],
        )
        quick_results.append({
            "model": model_name,
            **combo,
            "metrics": metrics
        })

# Display quick results
for res in quick_results:
    print(res)


Download complete
Download complete

Quick Training Falah/Alzheimer_classification_model with lr=0.0001


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/343M [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.222090,0.920000,0.916406


Quick Metrics for Falah/Alzheimer_classification_model with lr=5e-05: {'eval_loss': 0.22208969295024872, 'eval_accuracy': 0.92, 'eval_f1': 0.9164056006298996, 'eval_runtime': 72.5176, 'eval_samples_per_second': 1.379, 'eval_steps_per_second': 0.345, 'epoch': 1.0}

Quick Training Falah/Alzheimer_classification_model with lr=1e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-1-18ecd4c22a68>:100: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.226147,0.920000,0.916406


Quick Metrics for Falah/Alzheimer_classification_model with lr=1e-05: {'eval_loss': 0.22614705562591553, 'eval_accuracy': 0.92, 'eval_f1': 0.9164056006298996, 'eval_runtime': 75.7341, 'eval_samples_per_second': 1.32, 'eval_steps_per_second': 0.33, 'epoch': 1.0}
{'model': 'Falah/Alzheimer_classification_model', 'learning_rate': 0.0001, 'metrics': {'eval_loss': 0.2071635127067566, 'eval_accuracy': 0.94, 'eval_f1': 0.9274968053103946, 'eval_runtime': 71.2493, 'eval_samples_per_second': 1.404, 'eval_steps_per_second': 0.351, 'epoch': 1.0}}
{'model': 'Falah/Alzheimer_classification_model', 'learning_rate': 5e-05, 'metrics': {'eval_loss': 0.22208969295024872, 'eval_accuracy': 0.92, 'eval_f1': 0.9164056006298996, 'eval_runtime': 72.5176, 'eval_samples_per_second': 1.379, 'eval_steps_per_second': 0.345, 'epoch': 1.0}}
{'model': 'Falah/Alzheimer_classification_model', 'learning_rate': 1e-05, 'metrics': {'eval_loss': 0.22614705562591553, 'eval_accuracy': 0.92, 'eval_f1': 0.9164056006298996, 'eva

In [ ]:
import os
import requests
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoImageProcessor, AutoModelForImageClassification, Trainer, TrainingArguments
)
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score
from io import BytesIO
from torchvision import transforms

os.environ["WANDB_DISABLED"] = "true"

models_to_try = [
    "Falah/Alzheimer_classification_model"
]

urltest = "https://huggingface.co/datasets/Falah/Alzheimer_MRI/resolve/main/data/test-00000-of-00001-44110b9df98c5585.parquet"
urltrain = "https://huggingface.co/datasets/Falah/Alzheimer_MRI/resolve/main/data/train-00000-of-00001-c08a401c53fe5312.parquet"
output_file_test = "test_data.parquet"
output_file_train = "train_data.parquet"

def download_file(url, output_file):
    if not os.path.exists(output_file):
        print(f"Downloading {output_file}...")
        response = requests.get(url)
        with open(output_file, 'wb') as f:
            f.write(response.content)
        print("Download complete")
    else:
        print(f"{output_file} already exists.")

download_file(urltrain, output_file_train)
download_file(urltest, output_file_test)

data_train = pd.read_parquet(output_file_train)
data_test = pd.read_parquet(output_file_test)

class AlzheimerDataset(Dataset):
    def __init__(self, dataframe, processor):
        self.data = dataframe
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_data = self.data.iloc[idx]['image']['bytes']
        label = self.data.iloc[idx]['label']
        image = Image.open(BytesIO(image_data)).convert("RGB")
        encoding = self.processor(images=image, return_tensors="pt")
        return {
            "pixel_values": encoding["pixel_values"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

def subset_data(dataframe, size):
    return dataframe.sample(n=min(size, len(dataframe)))

def train_and_evaluate_quick(model_name, gradient_accumulation_step):
    print(f"\nQuick Training {model_name} with lr={gradient_accumulation_step}")

    processor = AutoImageProcessor.from_pretrained(model_name)

    train_subset = subset_data(data_train, 250)
    test_subset = subset_data(data_test, 100)

    train_dataset = AlzheimerDataset(train_subset, processor)
    test_dataset = AlzheimerDataset(test_subset, processor)

    model = AutoModelForImageClassification.from_pretrained(model_name, num_labels=4, ignore_mismatched_sizes=True)

    for param in model.base_model.parameters():
        param.requires_grad = False

    training_args = TrainingArguments(
        output_dir=f"./results_quick_{model_name.replace('/', '_')}",
        evaluation_strategy="epoch",
        save_strategy="no",
        learning_rate=1e-4,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        weight_decay=0.01,
        logging_dir=f"./logs_quick_{model_name.replace('/', '_')}",
        logging_steps=100,
        fp16=True,
        gradient_accumulation_steps=gradient_accumulation_step,
    )

    def compute_metrics(pred):
        labels = pred.label_ids
        preds = np.argmax(pred.predictions, axis=1)
        acc = accuracy_score(labels, preds)
        f1 = f1_score(labels, preds, average="weighted")
        return {"accuracy": acc, "f1": f1}

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=processor,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    metrics = trainer.evaluate(test_dataset)
    print(f"Quick Metrics for {model_name} with gras={gradient_accumulation_step}: {metrics}")
    return metrics

random_hyperparameter = [
    {"gradient_accumulation_step": 1},
    {"gradient_accumulation_step": 2},
    {"gradient_accumulation_step": 4},
]

quick_results = []

for model_name in models_to_try:
    for combo in random_hyperparameter:
        metrics = train_and_evaluate_quick(
            model_name,
            combo["gradient_accumulation_step"],
        )
        quick_results.append({
            "model": model_name,
            **combo,
            "metrics": metrics
        })

# Display quick results
for res in quick_results:
    print(res)


train_data.parquet already exists.
test_data.parquet already exists.

Quick Training Falah/Alzheimer_classification_model with lr=1


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-2-a4fb886f5940>:101: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.217797,0.920000,0.910810


Quick Metrics for Falah/Alzheimer_classification_model with gras=1: {'eval_loss': 0.2177971601486206, 'eval_accuracy': 0.92, 'eval_f1': 0.9108097928436912, 'eval_runtime': 70.3569, 'eval_samples_per_second': 1.421, 'eval_steps_per_second': 0.355, 'epoch': 1.0}

Quick Training Falah/Alzheimer_classification_model with lr=2


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-2-a4fb886f5940>:101: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,No log,0.221058,0.920000,0.916406


Quick Metrics for Falah/Alzheimer_classification_model with gras=2: {'eval_loss': 0.2210579663515091, 'eval_accuracy': 0.92, 'eval_f1': 0.9164056006298996, 'eval_runtime': 71.2697, 'eval_samples_per_second': 1.403, 'eval_steps_per_second': 0.351, 'epoch': 0.9841269841269841}

Quick Training Falah/Alzheimer_classification_model with lr=4


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-2-a4fb886f5940>:101: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,No log,0.224147,0.920000,0.916406


Quick Metrics for Falah/Alzheimer_classification_model with gras=4: {'eval_loss': 0.22414733469486237, 'eval_accuracy': 0.92, 'eval_f1': 0.9164056006298996, 'eval_runtime': 71.3852, 'eval_samples_per_second': 1.401, 'eval_steps_per_second': 0.35, 'epoch': 0.9523809523809523}
{'model': 'Falah/Alzheimer_classification_model', 'gradient_accumulation_step': 1, 'metrics': {'eval_loss': 0.2177971601486206, 'eval_accuracy': 0.92, 'eval_f1': 0.9108097928436912, 'eval_runtime': 70.3569, 'eval_samples_per_second': 1.421, 'eval_steps_per_second': 0.355, 'epoch': 1.0}}
{'model': 'Falah/Alzheimer_classification_model', 'gradient_accumulation_step': 2, 'metrics': {'eval_loss': 0.2210579663515091, 'eval_accuracy': 0.92, 'eval_f1': 0.9164056006298996, 'eval_runtime': 71.2697, 'eval_samples_per_second': 1.403, 'eval_steps_per_second': 0.351, 'epoch': 0.9841269841269841}}
{'model': 'Falah/Alzheimer_classification_model', 'gradient_accumulation_step': 4, 'metrics': {'eval_loss': 0.22414733469486237, 'eva

In [ ]:
import os
import requests
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoImageProcessor, AutoModelForImageClassification, Trainer, TrainingArguments
)
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score
from io import BytesIO
from torchvision import transforms

os.environ["WANDB_DISABLED"] = "true"

models_to_try = [
    "Falah/Alzheimer_classification_model"
]

urltest = "https://huggingface.co/datasets/Falah/Alzheimer_MRI/resolve/main/data/test-00000-of-00001-44110b9df98c5585.parquet"
urltrain = "https://huggingface.co/datasets/Falah/Alzheimer_MRI/resolve/main/data/train-00000-of-00001-c08a401c53fe5312.parquet"
output_file_test = "test_data.parquet"
output_file_train = "train_data.parquet"

def download_file(url, output_file):
    if not os.path.exists(output_file):
        print(f"Downloading {output_file}...")
        response = requests.get(url)
        with open(output_file, 'wb') as f:
            f.write(response.content)
        print("Download complete")
    else:
        print(f"{output_file} already exists.")

download_file(urltrain, output_file_train)
download_file(urltest, output_file_test)

data_train = pd.read_parquet(output_file_train)
data_test = pd.read_parquet(output_file_test)

class AlzheimerDataset(Dataset):
    def __init__(self, dataframe, processor):
        self.data = dataframe
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_data = self.data.iloc[idx]['image']['bytes']
        label = self.data.iloc[idx]['label']
        image = Image.open(BytesIO(image_data)).convert("RGB")
        encoding = self.processor(images=image, return_tensors="pt")
        return {
            "pixel_values": encoding["pixel_values"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

def subset_data(dataframe, size):
    return dataframe.sample(n=min(size, len(dataframe)))

def train_and_evaluate_quick(model_name, weight_decay):
    print(f"\nQuick Training {model_name} with wd={weight_decay}")

    processor = AutoImageProcessor.from_pretrained(model_name)

    train_subset = subset_data(data_train, 250)
    test_subset = subset_data(data_test, 100)

    train_dataset = AlzheimerDataset(train_subset, processor)
    test_dataset = AlzheimerDataset(test_subset, processor)

    model = AutoModelForImageClassification.from_pretrained(model_name, num_labels=4, ignore_mismatched_sizes=True)

    for param in model.base_model.parameters():
        param.requires_grad = False

    training_args = TrainingArguments(
        output_dir=f"./results_quick_{model_name.replace('/', '_')}",
        evaluation_strategy="epoch",
        save_strategy="no",
        learning_rate=1e-4,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        weight_decay=weight_decay,
        logging_dir=f"./logs_quick_{model_name.replace('/', '_')}",
        logging_steps=100,
        fp16=True,
        gradient_accumulation_steps=1,
    )

    def compute_metrics(pred):
        labels = pred.label_ids
        preds = np.argmax(pred.predictions, axis=1)
        acc = accuracy_score(labels, preds)
        f1 = f1_score(labels, preds, average="weighted")
        return {"accuracy": acc, "f1": f1}

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=processor,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    metrics = trainer.evaluate(test_dataset)
    print(f"Quick Metrics for {model_name} with wd={weight_decay}: {metrics}")
    return metrics

random_hyperparameter = [
    {"weight_decay": 0.005},
    {"weight_decay": 0.01},
    {"weight_decay": 0.05},
    {"weight_decay": 0.1},
]

quick_results = []

for model_name in models_to_try:
    for combo in random_hyperparameter:
        metrics = train_and_evaluate_quick(
            model_name,
            combo["weight_decay"],
        )
        quick_results.append({
            "model": model_name,
            **combo,
            "metrics": metrics
        })

# Display quick results
for res in quick_results:
    print(res)


train_data.parquet already exists.
test_data.parquet already exists.

Quick Training Falah/Alzheimer_classification_model with wd=0.005


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-3-f1ead0cff2e4>:101: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.217795,0.920000,0.910810


Quick Metrics for Falah/Alzheimer_classification_model with wd=0.005: {'eval_loss': 0.2177947610616684, 'eval_accuracy': 0.92, 'eval_f1': 0.9108097928436912, 'eval_runtime': 79.028, 'eval_samples_per_second': 1.265, 'eval_steps_per_second': 0.316, 'epoch': 1.0}

Quick Training Falah/Alzheimer_classification_model with wd=0.01


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-3-f1ead0cff2e4>:101: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.217816,0.920000,0.910810


Quick Metrics for Falah/Alzheimer_classification_model with wd=0.05: {'eval_loss': 0.21781620383262634, 'eval_accuracy': 0.92, 'eval_f1': 0.9108097928436912, 'eval_runtime': 72.0317, 'eval_samples_per_second': 1.388, 'eval_steps_per_second': 0.347, 'epoch': 1.0}

Quick Training Falah/Alzheimer_classification_model with wd=0.1


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-3-f1ead0cff2e4>:101: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.217840,0.920000,0.910810


Quick Metrics for Falah/Alzheimer_classification_model with wd=0.1: {'eval_loss': 0.2178400456905365, 'eval_accuracy': 0.92, 'eval_f1': 0.9108097928436912, 'eval_runtime': 71.3871, 'eval_samples_per_second': 1.401, 'eval_steps_per_second': 0.35, 'epoch': 1.0}
{'model': 'Falah/Alzheimer_classification_model', 'weight_decay': 0.005, 'metrics': {'eval_loss': 0.2177947610616684, 'eval_accuracy': 0.92, 'eval_f1': 0.9108097928436912, 'eval_runtime': 79.028, 'eval_samples_per_second': 1.265, 'eval_steps_per_second': 0.316, 'epoch': 1.0}}
{'model': 'Falah/Alzheimer_classification_model', 'weight_decay': 0.01, 'metrics': {'eval_loss': 0.2177971601486206, 'eval_accuracy': 0.92, 'eval_f1': 0.9108097928436912, 'eval_runtime': 69.5651, 'eval_samples_per_second': 1.438, 'eval_steps_per_second': 0.359, 'epoch': 1.0}}
{'model': 'Falah/Alzheimer_classification_model', 'weight_decay': 0.05, 'metrics': {'eval_loss': 0.21781620383262634, 'eval_accuracy': 0.92, 'eval_f1': 0.9108097928436912, 'eval_runtime'